## 1. Datos del Proceso de Admisión

En esta libreta se ejecuta la limpieza de los datos correspondientes a la información del proceso de admisión de la Univerdidad.

Primero, se leen todos los archivos correspondientes a los datos del proceso de admisión.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Límites de renglones y columnas a mostrar al imprimir DFs
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [2]:
# Se leen los archivos de admitidos
dfA_2012_2018 = pd.read_pickle('../dfsWithCorrectedSchema/procesoAdmision/dfA_2012_2018.pkl')
dfA_2019 = pd.read_pickle('../dfsWithCorrectedSchema/procesoAdmision/dfA_2019.pkl')
dfA_2020 = pd.read_pickle('../dfsWithCorrectedSchema/procesoAdmision/dfA_2020.pkl')
dfA_2021 = pd.read_pickle('../dfsWithCorrectedSchema/procesoAdmision/dfA_2021.pkl')
dfA_2022 = pd.read_pickle('../dfsWithCorrectedSchema/procesoAdmision/dfA_2022.pkl')

# Se leen los archivos de solicitudes de admisión
dfS_2012_2018 = pd.read_pickle('../dfsWithCorrectedSchema/dfS_2012_2018.pkl')
dfS_2019 = pd.read_pickle('../dfsWithCorrectedSchema/procesoAdmision/dfS_2019.pkl')
dfS_2020 = pd.read_pickle('../dfsWithCorrectedSchema/procesoAdmision/dfS_2020.pkl')
dfS_2021 = pd.read_pickle('../dfsWithCorrectedSchema/procesoAdmision/dfS_2021.pkl')

# Se almacenan todos los dataframes en un diccionario
dfProcAd = {
    2018:{'admitidos':dfA_2012_2018.copy(), 'solicitudesAdmision':dfS_2012_2018.copy()},
    2019:{'admitidos':dfA_2019.copy(),'solicitudesAdmision':dfS_2019.copy()},
    2020:{'admitidos':dfA_2020.copy(),'solicitudesAdmision':dfS_2020.copy()},
    2021:{'admitidos':dfA_2021.copy(),'solicitudesAdmision':dfS_2021.copy()},
    2022:{'admitidos':dfA_2022.copy()}
}

Se revisa el número de registros en cada dataframe

In [3]:
print('----- NUM. DE REGISTROS INICIALES EN LOS DATOS DEL PROCESO DE ADMISIÓN -----\n')
# Se almacenan los dataframes en una carpeta local
for anio in dfProcAd.keys():
    for tipo in dfProcAd[anio].keys():
        print(f'Num. de registros en el dataframe {tipo} del {anio}:', len(dfProcAd[anio][tipo]))

----- NUM. DE REGISTROS INICIALES EN LOS DATOS DEL PROCESO DE ADMISIÓN -----

Num. de registros en el dataframe admitidos del 2018: 50386
Num. de registros en el dataframe solicitudesAdmision del 2018: 171304
Num. de registros en el dataframe admitidos del 2019: 9173
Num. de registros en el dataframe solicitudesAdmision del 2019: 31173
Num. de registros en el dataframe admitidos del 2020: 8572
Num. de registros en el dataframe solicitudesAdmision del 2020: 39451
Num. de registros en el dataframe admitidos del 2021: 8150
Num. de registros en el dataframe solicitudesAdmision del 2021: 35484
Num. de registros en el dataframe admitidos del 2022: 9403


### 1.1. Uniformizar la columna `sexo`

En los archivos del proceso de admisión 2020 y 2021, así como en el archivo de aceptados 2022, la columna `sexo` contiene los valores 0 y 1, a diferencia de los periodos anteriores donde contiene las cadenas `HOMBRE` y `MUJER`. Hay que uniformizar esto en todos los conjuntos de datos.

In [4]:
def uniformizarColSexo(df):
    '''
        Esta función uniformiza los valores de la columna "sexo"
        
        Params
        ------
        * df <pandas.DataFrame>: Dataframe que contiene la columna "sexo" a uniformizar
        
        Returns
        -------
        * df <pandas.DataFrame>: Dataframe con la columna "sexo" uniformizada
    '''
    # Si la cadena 'HOMBRE' está presente, entonces la cadena 'MUJER' también lo estará.
    # En estos casos reemplazar "HOMBRE" con 1 y "MUJER"con 0.
    if 'HOMBRE' in df.sexo.unique():
        df.replace({'sexo':{'HOMBRE':1, 'MUJER':0}}, inplace=True)
    
    # Primero se aplica el método pd.to_numeric con el parámetro errors="coerce"
    # para que todos los valores que no sean ni 1, ni 0 se conviertan directamente a NaNs
    df.sexo = pd.to_numeric(df.sexo, errors='coerce')
    # Ahora, la columna se convierte a Int64, es decir, de tipo Int pero
    # que acepte valores nulos.
    df.sexo = df.sexo.astype('Int64')
    
    # Se regresa el df con la columna "sexo" actualizada
    return df

# Se aplica la función a todos los dataframes
for anio in dfProcAd.keys():
    for tipo in dfProcAd[anio].keys():
        dfProcAd[anio][tipo] = uniformizarColSexo(dfProcAd[anio][tipo])

### 1.2. Eliminación de las columnas relacionadas con el EXCOBA y EXHCOBA en los archivos que no las necesitan

Eliminar las columnas que miden el desempeño en EXCOBA y EXHCOBA en los archivos del proceso de admisión 2020, 2021 y archivo de aceptados 2022. Ya se revisó que todos los registros tienen NaN en esas columnas porque en estos periodos ya no se utilizaron dichos exámenes.

In [5]:
for anio in [2020, 2021, 2022]:
    for tipo in dfProcAd[anio].keys():
        dfProcAd[anio][tipo].drop(
            columns=[
                'verbAciertos','cuantiAciertos', 'espAciertos', 'matAciertos', 'natAciertos',
                'socialAciertos', 'asig1ExAd', 'asig1ExAdAciertos', 'asig2ExAd',
                'asig2ExAdAciertos', 'asig3ExAd', 'asig3ExAdAciertos'
            ],
            inplace=True
        )

### 1.3. Uniformizar la columna `resultadoFinal`

Como nos lo comentaron los expertos en el dominio del problema, desde el 2020 hasta la fecha todos los renglones que contienen valores nulos en las columnas correspondientes al examen de admisión (`totalExani2`, `califExamen`, `califExamenPonderado` y `promedioPrepaPonderado`) representan a alumnos que fueron aceptados directamente por su promedio del bachillerato.

Para estandarizar esta columna la castearemos a `float` y a los exentos se les asignará una calificación imposiblemente alta (**101**). Se corroboraron los valores máximos de esta columna desde el 2020 hasta la actualidad para elegir ese valor

Ademas, para los no exentos se dedujo el resultadoFinal sumando  columna califExamenPonderado y promedioPrepaPonderado. Este último paso se aplicó sobre todos los dataframes para asegurar que esta columna siempre tenga valores correctos 

In [6]:
def uniformizarColResultadoFinal(df):
    '''
        Esta función uniformiza los valores de la columna "resultadoFinal"
        
        Params
        ------
        * df <pandas.DataFrame>: Dataframe que contiene la columna "resultadoFinal" a uniformizar
        
        Returns
        -------
        * df <pandas.DataFrame>: Dataframe con la columna "resultadoFinal" uniformizada
    '''
    # La columna se vuelve numérica
    df.resultadoFinal = pd.to_numeric(df.resultadoFinal, errors='coerce')
    # Se deduce el resultado final de todas las columnas en las que existan datos del examen de admisión 
    df.resultadoFinal = df.califExamenPonderado + df.promedioPrepaPonderado
    # A los renglones que no tienen datos en las columnas correspondientes al examen de admisión se les asigna el valor 101 porque
    # han sido aceptados directamente 
    df.loc[df.totalExani2.isna()&df.califExamen.isna()&df.califExamenPonderado.isna()&df.promedioPrepaPonderado.isna(), 'resultadoFinal'] = 101
    
    return df

# Se aplica la función a todos los dataframes desde el 2020 a la actualidad
for anio in range(2020, 2022):
    for tipo in dfProcAd[anio].keys():
        dfProcAd[anio][tipo] = uniformizarColResultadoFinal(dfProcAd[anio][tipo])
dfProcAd[2022]['admitidos'] = uniformizarColResultadoFinal(dfProcAd[2022]['admitidos'])

# Se deduce el valor de la columna resultadoFinal en todos los demás dataframes
for anio in range(2018,2020):
    for tipo in dfProcAd[anio].keys():
        dfProcAd[anio][tipo].resultadoFinal = dfProcAd[anio][tipo].califExamenPonderado + dfProcAd[anio][tipo].promedioPrepaPonderado 

### 1.4. Corrección de inconsistencias en la columna `tipoExamenUnison`

Como ya nos lo habían comentado los expertos en el dominio del problema, existen discrepancias entre el tipo de examen aplicado a los estudiantes de nuevo ingreso (columna `tipoExamenUnison`) y las tres materias que deberían evaluarse en éstos (columnas `asig1ExAd`, `asig2ExAd` y `asig3ExAd`); es decir, las materias evaluadas no coinciden con su correspondiente examen. Por ese motivo nos facilitaron la relación Tipo examen - Materias que se ilustra a continuación

![](./imgs/relacionTipoExamenMaterias.png)

Esta relación corresponde a los periodos 2012-2019; sin embargo, en los archivos del 2020-2022 también aparece esta columna `tipoExamenUnison` y tiene valores no nulos. La estrategia actual es corregir los registros del 2012-2019 y, en base a esos datos, registrar qué tipo de exámenes se aplican en cuáles carreras y de esta forma corregir esta columna en los periodos 2020-2022 en caso de ser necesario.

Ya que las tres asignaturas pueden estar escritas de manera diferente en cada periodo, se necesitará inspeccionar manualmente los distintos valores posibles.

#### 1.4.1 Periodo 2012-2018

In [1]:
def removeAccents(s):
    '''
        Esta función remueve los acentos de una cadena de texto
        
        Params
        ------
        * s <str>: Cadena de texto con acentos
        
        Returns
        -------
        * s <str>: Cadena de texto sin acentos
    '''
    s = s.str.lower()
    s = s.str.replace('á','a',regex=False) 
    s = s.str.replace('í','i',regex=False) 
    s = s.str.replace('.','',regex=False) 

    return s

def obtenerTripletasAsigExAd(df):
    '''
        Esta función calcula las tripletas de asignaturas especiales evaluadas en el examen de admisión
        
        Params
        ------
        * df <pandas.DataFrame>: Dataframe que contiene las columnas 'asig1ExAd','asig2ExAd' y 'asig3ExAd'
        
        Returns
        -------
        * asignaturas <list>: Lista de las tripletas de asignaturas calculadas
    '''
    df[['asig1ExAd','asig2ExAd','asig3ExAd']] = df[['asig1ExAd','asig2ExAd','asig3ExAd']].apply(removeAccents)

    # Se toman todas las tripletas existentes de asignaturas evaluadas en el examen de
    # admisión
    asignaturas = [asig for asig in zip(df.loc[:,'asig1ExAd'],df.loc[:,'asig2ExAd'],df.loc[:,'asig3ExAd'])]

    asignaturas = list(set(asignaturas))
    asignaturas.sort()
    return asignaturas

In [8]:
tripletasAdmitidos = obtenerTripletasAsigExAd(dfProcAd[2018]['admitidos'])
tripletasSolicitudes = obtenerTripletasAsigExAd(dfProcAd[2018]['solicitudesAdmision'])

print('Tripletas (Admitidos)')
for t in tripletasAdmitidos:
    print(t)

print('\nTripletas (Solicitudes)')
for t in tripletasSolicitudes:
    print(t)

print(f'\nLas tripletas son iguales en ambos archivos?: {tripletasAdmitidos==tripletasSolicitudes}')

Tripletas (Admitidos)
('fisica', 'biologia', 'quimica')
('humanidades', 'cs sociales', 'lenguaje')
('mat calculo', 'biologia', 'quimica')
('mat calculo', 'fisica', 'lenguaje')
('mat calculo', 'fisica', 'quimica')
('mat estad', 'cs sociales', 'econ admivas')
('mat estad', 'cs sociales', 'lenguaje')

Tripletas (Solicitudes)
('fisica', 'biologia', 'quimica')
('humanidades', 'cs sociales', 'lenguaje')
('mat calculo', 'biologia', 'quimica')
('mat calculo', 'fisica', 'lenguaje')
('mat calculo', 'fisica', 'quimica')
('mat estad', 'cs sociales', 'econ admivas')
('mat estad', 'cs sociales', 'lenguaje')

Las tripletas son iguales en ambos archivos?: True


In [9]:
def corregirTipoExamen(df, tripletas, numsExamenesOrdenados):
    '''
        Esta función corrige el tipo de examen aplicado de acuerdo a las asignaturas
        especiales detectadas para cada registro
        
        Params
        ------
        * df <pandas.DataFrame>: Dataframe que contiene la columna "tipoExamenUnison" a corregir
        * tripletas <list>: Lista de las tripletas de asignaturas especiales evaluadas en el examen de admisión
        * numsExamenesOrdenados <list>: Lista que indica el examen correspondiente a cada tripleta
        
        Returns
        -------
        * df <pandas.DataFrame>: Dataframe con la columna "tipoExamenUnison" corregida
    '''
    # Se crea un diccionario donde las llaves son las tripletas de asignaturas y los valores son
    # el número del examen correspondiente
    tripletasAsignaturas = dict(zip(tripletas,numsExamenesOrdenados))
    # Se encuentran todos los renglones correspondientes a cada tripleta y se le asigna su valor
    for t in tripletasAsignaturas.keys():
        df.loc[((df.asig1ExAd==t[0])&(df.asig2ExAd==t[1])&(df.asig3ExAd==t[2])), 'tipoExamenUnison'] = tripletasAsignaturas[t]
    
    df.tipoExamenUnison = df.tipoExamenUnison.astype('str')
    return df

dfProcAd[2018]['admitidos'] = corregirTipoExamen(dfProcAd[2018]['admitidos'],tripletasAdmitidos,[3,6,2,5,4,1,7])
dfProcAd[2018]['solicitudesAdmision'] = corregirTipoExamen(dfProcAd[2018]['solicitudesAdmision'],tripletasSolicitudes,[3,6,2,5,4,1,7])

#### 1.4.2. Pedriodo 2019

In [10]:
tripletasAdmitidos = obtenerTripletasAsigExAd(dfProcAd[2019]['admitidos'])
tripletasSolicitudes = obtenerTripletasAsigExAd(dfProcAd[2019]['solicitudesAdmision'])

print('Tripletas (Admitidos)')
for t in tripletasAdmitidos:
    print(t)

print('\nTripletas (Solicitudes)')
for t in tripletasSolicitudes:
    print(t)

print(f'\nLas tripletas son iguales en ambos archivos?: {tripletasAdmitidos==tripletasSolicitudes}')

Tripletas (Admitidos)
('0', '0', '0')
('fisica', 'biologia', 'quimica')
('humanidades', 'cs sociales', 'lenguaje')
('mat calculo', 'biologia', 'quimica')
('mat calculo', 'fisica', 'lenguaje')
('mat calculo', 'fisica', 'quimica')
('mat estad', 'cs sociales', 'econ admivas')
('mat estad', 'cs sociales', 'lenguaje')

Tripletas (Solicitudes)
('0', '0', '0')
('fisica', 'biologia', 'quimica')
('humanidades', 'cs sociales', 'lenguaje')
('mat calculo', 'biologia', 'quimica')
('mat calculo', 'fisica', 'lenguaje')
('mat calculo', 'fisica', 'quimica')
('mat estad', 'cs sociales', 'econ admivas')
('mat estad', 'cs sociales', 'lenguaje')

Las tripletas son iguales en ambos archivos?: True


En este periodo vemos que existen los valores **0** en algunos renglones. Analizaremos este hecho:

In [11]:
d = dfProcAd[2019]['admitidos']
d.loc[(d.asig1ExAd=='0')&(d.asig2ExAd=='0')&(d.asig3ExAd=='0')].head()

,campus,division,cvePrograma,ciclo,numExpediente,paisOrigen,estadoOrigen,ciudadOrigen,esMexicano,sexo,tieneDiscapacidad,descripcionDiscapacidad,alumnoUnison,empleado,dependiente,descripcionGpoEtnico,sistemaPrepa,clavePrepa,nombrepPrepa,anioIngresoPrepa,anioEgresoPrepa,promedioPrepa,areaEstudioPrepa,modalidadEstudioPrepa,idSolicitudAspirante,tipoExamenUnison,verbAciertos,cuantiAciertos,espAciertos,matAciertos,natAciertos,socialAciertos,asig1ExAd,asig1ExAdAciertos,asig2ExAd,asig2ExAdAciertos,asig3ExAd,asig3ExAdAciertos,totalAciertos,califExamen,califExamenPonderado,promedioPrepaPonderado,resultadoFinal,nivelInglesAcreditado,nombrePrograma,unidadRegional
107,HERMOSILLO,DIVISIÓN DE INGENIERÍA,LSU,2019,219213225,MEXICO,SONORA,HERMOSILLO,1,1,0,NaN,0,0,0,NO DISPONIBLE,COBACH (COLEGIO DE BACHILLERES),26XCB0004J,COBACH VILLA DE SERIS HERMOSILLO VIII,1998,2001,76.00,CONSTRUCCIÓN,ESCOLARIZADA,366241,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,100.0,60.0,30.400,90.400,NaN,LICENCIATURA EN SUSTENTABILIDAD,CENTRO
544,HERMOSILLO,DIVISIÓN DE INGENIERÍA,LSU,2019,219217186,MEXICO,CHIHUAHUA,OCAMPO,1,1,0,NaN,0,0,0,NO DISPONIBLE,BACHILLERATO GENERAL 3 AÑOS,16NTA0001Q,CENTRO DE EDUCACION Y CAPACITACION FORESTAL NO...,1990,1993,86.45,OTRAS,ABIERTA,375040,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,100.0,60.0,34.580,94.580,NaN,LICENCIATURA EN SUSTENTABILIDAD,CENTRO
741,HERMOSILLO,DIVISIÓN DE INGENIERÍA,LSU,2019,219218268,MEXICO,SONORA,GUAYMAS,1,0,0,NaN,1,0,0,NO DISPONIBLE,CBTIS/CETIS/CECYT (C. DE BACH. IND. Y DE SERV),26DCT0040Q,CENTRO DE BACHILLERATO TECNOLÓGICO INDUSTRIAL ...,1994,1997,82.00,ECO-ADMVAS,ESCOLARIZADA,377025,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,100.0,60.0,32.800,92.800,NaN,LICENCIATURA EN SUSTENTABILIDAD,CENTRO
1182,HERMOSILLO,DIVISIÓN DE INGENIERÍA,LSU,2019,219220301,MEXICO,SONORA,CABORCA,1,0,0,NaN,0,0,0,NO DISPONIBLE,COBACH (COLEGIO DE BACHILLERES),26ECB0005U,COLEGIO DE BACHILLERES DE CABORCA,1990,1993,76.42,OTRAS,ESCOLARIZADA,381928,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,100.0,60.0,30.568,90.568,NaN,LICENCIATURA EN SUSTENTABILIDAD,CENTRO
1472,HERMOSILLO,DIVISIÓN DE INGENIERÍA,LSU,2019,219201432,MEXICO,SONORA,HERMOSILLO,1,0,0,NaN,0,0,0,NO DISPONIBLE,CECYTES (SOLO ESTADO DE SONORA),26ETC0018N,CECYTES JUSTO SIERRA CECYTES IV CECYTES HERMO...,2011,2014,76.00,INFORMÁTICA,ESCOLARIZADA,382148,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,100.0,60.0,30.400,90.400,NaN,LICENCIATURA EN SUSTENTABILIDAD,CENTRO


In [12]:
d = dfProcAd[2019]['solicitudesAdmision']
d.loc[(d.asig1ExAd=='0')&(d.asig2ExAd=='0')&(d.asig3ExAd=='0')].head()

,campus,division,cvePrograma,ciclo,idSolicitudAspirante,paisOrigen,estadoOrigen,ciudadOrigen,esMexicano,sexo,tieneDiscapacidad,descripcionDiscapacidad,alumnoUnison,empleado,dependiente,descripcionGpoEtnico,sistemaPrepa,clavePrepa,nombrePrepa,anioIngresoPrepa,anioEgresoPrepa,promedioPrepa,areaEstudioPrepa,modalidadEstudioPrepa,tipoExamenUnison,verbAciertos,cuantiAciertos,espAciertos,matAciertos,natAciertos,socialAciertos,asig1ExAd,asig1ExAdAciertos,asig2ExAd,asig2ExAdAciertos,asig3ExAd,asig3ExAdAciertos,totalAciertos,califExamen,califExamenPonderado,promedioPrepaPonderado,resultadoFinal,nivelInglesAcreditado,nombrePrograma,unidadRegional
354,HERMOSILLO,DIVISIÓN DE INGENIERÍA,LSU,2019,366241,MEXICO,SONORA,HERMOSILLO,1,1,0,NaN,0,0,0,NO DISPONIBLE,COBACH (COLEGIO DE BACHILLERES),26XCB0004J,COBACH VILLA DE SERIS HERMOSILLO VIII,1998,2001,76.00,CONSTRUCCIÓN,ESCOLARIZADA,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,100.0,60.0,30.400,90.400,NaN,LICENCIATURA EN SUSTENTABILIDAD,CENTRO
1825,HERMOSILLO,DIVISIÓN DE INGENIERÍA,LSU,2019,375040,MEXICO,CHIHUAHUA,OCAMPO,1,1,0,NaN,0,0,0,NO DISPONIBLE,BACHILLERATO GENERAL 3 AÑOS,16NTA0001Q,CENTRO DE EDUCACION Y CAPACITACION FORESTAL NO...,1990,1993,86.45,OTRAS,ABIERTA,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,100.0,60.0,34.580,94.580,NaN,LICENCIATURA EN SUSTENTABILIDAD,CENTRO
2535,HERMOSILLO,DIVISIÓN DE INGENIERÍA,LSU,2019,377025,MEXICO,SONORA,GUAYMAS,1,0,0,NaN,1,0,0,NO DISPONIBLE,CBTIS/CETIS/CECYT (C. DE BACH. IND. Y DE SERV),26DCT0040Q,CENTRO DE BACHILLERATO TECNOLÓGICO INDUSTRIAL ...,1994,1997,82.00,ECO-ADMVAS,ESCOLARIZADA,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,100.0,60.0,32.800,92.800,NaN,LICENCIATURA EN SUSTENTABILIDAD,CENTRO
3955,HERMOSILLO,DIVISIÓN DE INGENIERÍA,LSU,2019,381928,MEXICO,SONORA,CABORCA,1,0,0,NaN,0,0,0,NO DISPONIBLE,COBACH (COLEGIO DE BACHILLERES),26ECB0005U,COLEGIO DE BACHILLERES DE CABORCA,1990,1993,76.42,OTRAS,ESCOLARIZADA,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,100.0,60.0,30.568,90.568,NaN,LICENCIATURA EN SUSTENTABILIDAD,CENTRO
4928,HERMOSILLO,DIVISIÓN DE INGENIERÍA,LSU,2019,382148,MEXICO,SONORA,HERMOSILLO,1,0,0,NaN,0,0,0,NO DISPONIBLE,CECYTES (SOLO ESTADO DE SONORA),26ETC0018N,CECYTES JUSTO SIERRA CECYTES IV CECYTES HERMO...,2011,2014,76.00,INFORMÁTICA,ESCOLARIZADA,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,100.0,60.0,30.400,90.400,NaN,LICENCIATURA EN SUSTENTABILIDAD,CENTRO


In [13]:
d = dfProcAd[2019]['admitidos']
print(d.loc[(d.asig1ExAd=='0')&(d.asig2ExAd=='0')&(d.asig3ExAd=='0')].nombrePrograma.unique())
print(d.loc[(d.asig1ExAd=='0')&(d.asig2ExAd=='0')&(d.asig3ExAd=='0')].califExamen.unique())
print(len(d.loc[(d.asig1ExAd=='0')&(d.asig2ExAd=='0')&(d.asig3ExAd=='0')]))

d = dfProcAd[2019]['solicitudesAdmision']
print(d.loc[(d.asig1ExAd=='0')&(d.asig2ExAd=='0')&(d.asig3ExAd=='0')].nombrePrograma.unique())
print(d.loc[(d.asig1ExAd=='0')&(d.asig2ExAd=='0')&(d.asig3ExAd=='0')].califExamen.unique())
print(len(d.loc[(d.asig1ExAd=='0')&(d.asig2ExAd=='0')&(d.asig3ExAd=='0')]))

['LICENCIATURA EN SUSTENTABILIDAD']
[100.]
39
['LICENCIATURA EN SUSTENTABILIDAD']
[100.]
41


Podemos notar que existen 39 registros así en el archivo de admitidos, y 41 en el archivo de solicitudes. Todos corresponden a la licenciatura en sustentabilidad y muestran una puntuación perfecta en el examen de admisión; sin embargo, su tipo de examen es **NaN**. Al parecer no hicieron examen de admisión, ¿por qué?. Si revisamos los archivos del periodo anterior podemos observar que en esta carrera se había aplicado el examen tipo 4 anteriormente

In [14]:
d = dfProcAd[2018]['admitidos']
print(d[d.nombrePrograma == 'LICENCIATURA EN SUSTENTABILIDAD'].tipoExamenUnison.unique())

d = dfProcAd[2018]['solicitudesAdmision']
print(d[d.nombrePrograma == 'LICENCIATURA EN SUSTENTABILIDAD'].tipoExamenUnison.unique())

['4']
['4']


Sea cual sea la razón de este hecho dejaremos como **NaN** los valores del tipo de examen en estos casos

In [15]:
dfProcAd[2019]['admitidos'] = corregirTipoExamen(dfProcAd[2019]['admitidos'],tripletasAdmitidos,[np.nan,3,6,2,5,4,1,7])
dfProcAd[2019]['solicitudesAdmision'] = corregirTipoExamen(dfProcAd[2019]['solicitudesAdmision'],tripletasSolicitudes,[np.nan,3,6,2,5,4,1,7])

#### 1.4.3. Periodos 2020, 2021 y 2022

Como anteriormente se verificó que a partir del 2020 las columnas con el nombre de las asignaturas siempre estaban vacías ya se eliminaron; sin embargo, la columna `tipoExamenUnison` sigue presente y poblada. Para estandarizar los valores de esta columna en los periodos del 2020 en adelante, se utilizarán los tipos de exámenes corregidos del periodo inmediato anterior

In [16]:
def corregirTipoExamenReciente(periodoActual, periodoAnterior):
    '''
        Esta función corrige el tipo de examen de los periodos del 2020 al 2022
        
        Params
        ------
        * periodoActual <int>: Año correspondiente al periodo a corregir
        * periodoAnterior <int>: Año correspondiente al periodo anterior al que se pretende corregir
        
        Returns
        -------
        * None
    '''
    # Se toman todas las posibles parejas de cvePrograma - tipoExamenUnison del periodo anterior
    aAnterior = dfProcAd[periodoAnterior]['admitidos'].drop_duplicates(subset='cvePrograma').loc[:,['cvePrograma', 'tipoExamenUnison']]
    sAnterior = dfProcAd[periodoAnterior]['solicitudesAdmision'].drop_duplicates(subset='cvePrograma').loc[:,['cvePrograma', 'tipoExamenUnison']]
    totalAnterior = pd.concat([aAnterior,sAnterior])
    totalAnterior = totalAnterior.drop_duplicates()

    # Se toman todas las claves de programas posibles del periodo actual
    aActual = dfProcAd[periodoActual]['admitidos'].drop_duplicates(subset='cvePrograma').loc[:,['cvePrograma', 'tipoExamenUnison']]
    try:
        sActual = dfProcAd[periodoActual]['solicitudesAdmision'].drop_duplicates(subset='cvePrograma').loc[:,['cvePrograma', 'tipoExamenUnison']]
        totalActual = pd.concat([aActual,sActual])
    except:
        totalActual = aActual

    totalActual = totalActual.drop_duplicates()

    # Se hace un left join (para preservar todos los posibles valores de claves del periodo actual) entre la info del periodo actual y el
    # periodo anterior para obtener el tipo de examen de cada carrera
    j = totalActual.merge(totalAnterior, on='cvePrograma', how='left').sort_values(by='cvePrograma')

    # Se crea una nueva columna con el tipo de examen corregido. Esto es necesario ya que algunas carreras no tenían un examen asignado
    # en el periodo anterior pero en el actual sí lo tiene. Confiaremos en que esta nueva asignación es correcta. Además, si sigue
    # sin asignarse un examen, entonces el valor será NaN.
    j['tipoExamenCorregido'] = j.tipoExamenUnison_y.fillna(j.tipoExamenUnison_x).replace({'nan':np.nan}).str[-1]

    # Se crea un diccionario en donde las llaves son las claves de los programas y los valores son los tipos de examenes corregido
    tipoExamenesCorregidos = dict(zip(j.cvePrograma, j.tipoExamenCorregido))
    
    # Se realiza la correccion en el archivo de admitidos y de solicitudes
    for cve in tipoExamenesCorregidos.keys():
        dfProcAd[periodoActual]['admitidos'].loc[dfProcAd[periodoActual]['admitidos'].cvePrograma == cve, 'tipoExamenUnison'] = tipoExamenesCorregidos[cve]
        dfProcAd[periodoActual]['admitidos'].tipoExamenUnison.replace({'nan':np.nan})
        try:
            dfProcAd[periodoActual]['solicitudesAdmision'].loc[dfProcAd[periodoActual]['solicitudesAdmision'].cvePrograma == cve, 'tipoExamenUnison'] = tipoExamenesCorregidos[cve]
            dfProcAd[periodoActual]['solicitudesAdmision'].tipoExamenUnison.replace({'nan':np.nan})
        except:
            None

In [17]:
# Se aplica la corrección a los periodos 2020, 2021 y 2022
corregirTipoExamenReciente(2020, 2019)
corregirTipoExamenReciente(2021, 2020)
corregirTipoExamenReciente(2022, 2021)

for a in range(2020, 2023):
    print(f'tipos de examen archivo de admitidos {a}')
    print(dfProcAd[a]['admitidos'].tipoExamenUnison.unique())
    try:
        print(f'\ntipos de examen archivo de solicitudes {a}')
        print(dfProcAd[a]['solicitudesAdmision'].tipoExamenUnison.unique())
    except:
        None
    print('----------------------------------------------')


tipos de examen archivo de admitidos 2020
['3' '7' '2' '6' '4' '1' '5' nan]

tipos de examen archivo de solicitudes 2020
['3' '4' '7' '2' '1' '6' '5' nan]
----------------------------------------------
tipos de examen archivo de admitidos 2021
['1' '7' '4' '6' '5' '3' '2' nan]

tipos de examen archivo de solicitudes 2021
['1' '7' '4' '6' '5' '3' '2' nan]
----------------------------------------------
tipos de examen archivo de admitidos 2022
['2' '7' '4' nan '1' '6' '3' '5']

tipos de examen archivo de solicitudes 2022
----------------------------------------------


### 1.5. Corrección de los aciertos

En los periodos en donde se aplicó el EXCOBA  y EXHCOBA se reportan los aciertos que cada aspirante obtuvo en las diferentes asignaturas evaluadas (del 2012 al 2018 también se reportan los errores, los no sé y los no conestó). En una etapa anterior se descubrió que la sumatoria de estos aciertos no coincide con el número reportado en la columna `totalAciertos` en los archivos del 2019; sin embargo, este problema no se verificó en los archivos anteriores. Cabe destacar que a partir del 2020 únicamente se reporta el puntaje total del EXANI-II y la calificación final del examen de admisión; por lo tanto, este problema no ocurre en dichos periodos.

A continuación se verifica el problema del conteo del total de aciertos en todos los archivos

In [18]:
def verificarSumasConteo(d,tipoCol):
    '''
        Esta función verifica si el conteo de preguntas clasificadas como
        aciertos/errores/no sé/no contestó del examen de admisión coinciden con
        la columna que reporta el total de éstas
        
        Params
        ------
        * d <pandas.DataFrame>: Dataframe en el que se pretende verificar los conteos
        * tipoCol <str>: Tipo de preguntas a verificar
        
        Returns
        -------
        * <bool>: Valor booleano que indica si la sumatoria es correcta
    '''
    if tipoCol != 'NoSe':
        sumatoria = d.loc[:, [c for c in d.columns if c.endswith(tipoCol) and c != f'total{tipoCol}' and c != f'suma{tipoCol}']].sum(axis=1)
    else:
        sumatoria = d.loc[:, [c for c in d.columns if (c.endswith(tipoCol) or c.endswith('NoSabe')) and c != f'total{tipoCol}' and c != f'suma{tipoCol}']].sum(axis=1)
    return sumatoria.astype('Int64').equals(d.loc[:,f'total{tipoCol}'].astype('Int64'))

In [19]:
for a in range(2018,2020):
    for tipo in dfProcAd[a].keys():
        print(f'archivo {tipo} {a}')
        print(f'las sumatorias de aciertos coinciden?: {verificarSumasConteo(dfProcAd[a][tipo], "Aciertos")}')
        print('--------------------\n')

archivo admitidos 2018
las sumatorias de aciertos coinciden?: True
--------------------

archivo solicitudesAdmision 2018
las sumatorias de aciertos coinciden?: True
--------------------

archivo admitidos 2019
las sumatorias de aciertos coinciden?: False
--------------------

archivo solicitudesAdmision 2019
las sumatorias de aciertos coinciden?: False
--------------------



Para los datos del 2012-2018 el problema del conteo del total de aciertos no ocurre; sin embargo, en estos archivos también se reportan los conteos de errores, no sé y no contestó. Ahora se verificará la sumatoria de dichas variables

In [20]:
for tipoArchivo in ['admitidos', 'solicitudesAdmision']:
    print(f'{tipoArchivo} 2012-2018')
    for tipoConteo in ['Errores','NoSe','NoContesto']:
        print(f'sumatorias {tipoConteo} correcta?: ',verificarSumasConteo(dfProcAd[2018][tipoArchivo],tipoConteo))
    print('-----------------------')


admitidos 2012-2018
sumatorias Errores correcta?:  True
sumatorias NoSe correcta?:  True
sumatorias NoContesto correcta?:  True
-----------------------
solicitudesAdmision 2012-2018
sumatorias Errores correcta?:  True
sumatorias NoSe correcta?:  True
sumatorias NoContesto correcta?:  True
-----------------------


No se encontraron errores de conteo para los datos del 2012-2018.

Ahora, se corrigen los conteos en lo archivos del 2019

In [21]:
def corregirSumatoriaAciertos(d):
    '''
        Esta función corrige la sumatoria de las preguntas del examen de admisión
        clasificadas como Aciertos
        
        Params
        ------
        * d <pandas.DataFrame>: Dataframe en el que se pretende corregir los conteos de aciertos
        
        Returns
        -------
        * None
    '''
    sumatoriaAciertos = d.loc[:, [c for c in d.columns if c.endswith('Aciertos') and c != 'totalAciertos' and c != f'sumaAciertos']].sum(axis=1)
    d.totalAciertos = sumatoriaAciertos.astype('Int64')
    return d

dfProcAd[2018]['admitidos'] = corregirSumatoriaAciertos(dfProcAd[2018]['admitidos'])
dfProcAd[2018]['solicitudesAdmision'] = corregirSumatoriaAciertos(dfProcAd[2018]['solicitudesAdmision'])

print(verificarSumasConteo(dfProcAd[2018]['admitidos'], 'Aciertos'))
print(verificarSumasConteo(dfProcAd[2018]['solicitudesAdmision'], 'Aciertos'))

True
True


In [22]:
for anio in dfProcAd.keys():
    print(anio)
    for tipo in dfProcAd[anio].keys():
        d = dfProcAd[anio][tipo]
        print(tipo)
        nivIngles = list(d.nivelInglesAcreditado.unique())
        nivIngles.sort() 
        print(f'Valores en la columna nivelInglesAcreditado: {nivIngles}')
        print(f'Num. de renglones con nans en la columna NivelInglesAcreditado: {d.nivelInglesAcreditado.isna().sum()}')
        print('---------------------------------------------------')

2018
admitidos
Valores en la columna nivelInglesAcreditado: [0, 1, 2, 3, 4, 5, 6, 7]
Num. de renglones con nans en la columna NivelInglesAcreditado: 0
---------------------------------------------------
solicitudesAdmision
Valores en la columna nivelInglesAcreditado: [-1, 0, 1, 2, 3, 4, 5, 6, 7]
Num. de renglones con nans en la columna NivelInglesAcreditado: 0
---------------------------------------------------
2019
admitidos
Valores en la columna nivelInglesAcreditado: [0.0, 0.5, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, nan, 7.0]
Num. de renglones con nans en la columna NivelInglesAcreditado: 609
---------------------------------------------------
solicitudesAdmision
Valores en la columna nivelInglesAcreditado: [5.0, nan, 0.0, 0.5, 1.0, 2.0, 3.0, 4.0, 6.0, 7.0]
Num. de renglones con nans en la columna NivelInglesAcreditado: 12928
---------------------------------------------------
2020
admitidos
Valores en la columna nivelInglesAcreditado: [nan, 0.0, 0.5, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
Num. 

Se revisa el número de registros finales

In [23]:
print('----- NUM. DE REGISTROS FINALES EN LOS DATOS DEL PROCESO DE ADMISIÓN -----\n')
# Se almacenan los dataframes en una carpeta local
for anio in dfProcAd.keys():
    for tipo in dfProcAd[anio].keys():
        print(f'Num. de registros en el dataframe {tipo} del {anio}:', len(dfProcAd[anio][tipo]))

----- NUM. DE REGISTROS FINALES EN LOS DATOS DEL PROCESO DE ADMISIÓN -----

Num. de registros en el dataframe admitidos del 2018: 50386
Num. de registros en el dataframe solicitudesAdmision del 2018: 171304
Num. de registros en el dataframe admitidos del 2019: 9173
Num. de registros en el dataframe solicitudesAdmision del 2019: 31173
Num. de registros en el dataframe admitidos del 2020: 8572
Num. de registros en el dataframe solicitudesAdmision del 2020: 39451
Num. de registros en el dataframe admitidos del 2021: 8150
Num. de registros en el dataframe solicitudesAdmision del 2021: 35484
Num. de registros en el dataframe admitidos del 2022: 9403


Como ya se corrigieron todos los detalles en los archivos del proceso de nuevo ingreso se procede a guardarlos en formato .pkl

In [24]:
# Se almacenan los dataframes en una carpeta local
for anio in dfProcAd.keys():
    for tipo in dfProcAd[anio].keys():
        a = anio if anio != 2018 else '2012_'+str(anio)
        dfProcAd[anio][tipo].to_pickle(f'../correctedDfs/procesoAdmision/df{tipo[0].capitalize()}_{a}.pkl')